<h1>
    The Variational Approximation for Bayesian Inference
</h1>
<p>
    Life after the EM Algorithm
</p>
<br>
<p>
    Dimitris G. Tzikas, Aristidis C. Likas, and Nikolaos P. Galatsanos`
</p>
<br>
<br>
<br>
<br>
<br>
<p style="font-size:10px;">CMI Lab Journal Club, 11/15/2019, Andrew Van</p>

<p>This subslide runs a bunch of scripts for this presentation.</p>
<script>window.addEventListener('load', () => { Reveal.configure({ transition: 'fade' }); });</script>
<script></script>

<h1><center>Motivation</center></h1>


<h1><center>Overview</center></h1>


<h1><center>Introduction</center></h1>
